In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import h5py
import sklearn.metrics

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
img_size=224
train_generator = ImageDataGenerator(horizontal_flip = True).flow_from_directory('/content/drive/My Drive/mentorship2020/dataset/train',
     target_size=(img_size,img_size), 
     color_mode="rgb", batch_size=128,
     class_mode='categorical', 
     shuffle=True, 
                                                   )
TRAIN_DIR='/content/drive/My Drive/mentorship2020/dataset/train'
datagen = ImageDataGenerator(validation_split=0.20)
val_generator = datagen.flow_from_directory(
    TRAIN_DIR,
    subset='validation',
    target_size=(img_size,img_size), 
    color_mode="rgb", batch_size=128,
    class_mode='categorical', 
    shuffle=True 
)

type(train_generator)
type(train_generator)

Found 28787 images belonging to 7 classes.
Found 5754 images belonging to 7 classes.


tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [ ]:
IMAGE_SIZE = [224, 224]
vgg16_model = tf.keras.applications.vgg16.VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
model = keras.Sequential()

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
x = tf.keras.layers.Flatten()(vgg16_model.output)
x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.50)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
prediction = tf.keras.layers.Dense(7, activation='softmax')(x)
model = tf.keras.Model(inputs=vgg16_model.input, outputs=prediction)

In [ ]:
model.compile( loss='categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)
from tensorflow.keras.callbacks import ReduceLROnPlateau

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                           patience=3,
                                           verbose=1,
                                           factor=0.5,
                                           min_lr=0.00001)

In [ ]:
model.save("/content/drive/My Drive/model125epoch/mymodel200.h5")

In [ ]:
model.fit_generator(train_generator,
                  steps_per_epoch = 225,
                  epochs = 200,
                  #callbacks=[cp_callback],
                  validation_data = val_generator,
                  validation_steps = 45,)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/200
225/225 [==============================] - 12700s 56s/step - loss: 2.3656 - accuracy: 0.2357 - val_loss: 1.8124 - val_accuracy: 0.2555
Epoch 2/200
225/225 [==============================] - 117s 522ms/step - loss: 1.8077 - accuracy: 0.2507 - val_loss: 1.7842 - val_accuracy: 0.2614
Epoch 3/200
225/225 [==============================] - 118s 524ms/step - loss: 1.8031 - accuracy: 0.2520 - val_loss: 1.7761 - val_accuracy: 0.2638
Epoch 4/200
225/225 [==============================] - 119s 530ms/step - loss: 1.8019 - accuracy: 0.2545 - val_loss: 1.7965 - val_accuracy: 0.2548
Epoch 5/200
225/225 [==============================] - 117s 521ms/step - loss: 1.8076 - accuracy: 0.2521 - val_loss: 1.7850 - val_accuracy: 0.2626
Epoch 6/200
225/225 [==============================] - 117s 520ms/step - loss: 1.7884 - accuracy: 0.2595 - val_loss: 1.7486 - val_accuracy: 0.2722
Epoch 7/200
225/225 [=====================

In [ ]:
model.save("/content/drive/My Drive/model125epoch/mymodel200.h5")

In [ ]:
img_size=224
test_generator = ImageDataGenerator( ).flow_from_directory('/content/drive/My Drive/mentorship2020/dataset/train',
                                                   target_size=(img_size,img_size), 
                                                   color_mode="rgb",
                                                    shuffle=False,
                                                    #
                                                    classes=['disgust'],
                                                   class_mode=None, 
                                                   )

Found 435 images belonging to 1 classes.


In [ ]:
predictions = model.predict(
    test_generator,
    batch_size=32  ,
    verbose=1,
    steps=221,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)

171/221 [======================>.......] - ETA: 11:38

In [ ]:
print(predictions)

[[1.3614195e-17 2.2108889e-25 1.1611250e-11 ... 3.5201964e-14
  4.5986753e-15 1.0000000e+00]
 [1.1060366e-25 0.0000000e+00 1.5449435e-20 ... 8.0180261e-29
  5.7705869e-29 1.0000000e+00]
 [4.3562756e-10 4.4478462e-12 2.8414644e-05 ... 6.3723973e-09
  7.3033946e-10 9.9997151e-01]
 ...
 [1.7987071e-02 1.8578184e-01 3.0845615e-01 ... 6.6234702e-03
  5.6260757e-02 3.6405258e-02]
 [9.5147997e-01 4.4338913e-03 4.0489141e-02 ... 2.4201655e-04
  2.5987995e-03 6.3937873e-04]
 [9.8620185e-05 9.9981982e-01 3.0328337e-07 ... 5.4529931e-10
  1.6035176e-07 8.7065580e-08]]


In [ ]:
import pandas as pd
import numpy as np

prediction = pd.DataFrame(np.array(test_generator)[indices.astype(int)] , columns=['predictions']).to_csv('prediction2.csv')

NameError: ignored